# Proposed Methods
From our motivating experiments, we observed decreasing entropy and increasing EOS token probability in the blocks leading up to halted output. We also observed certain token combinations that may heavily increase the chance of EOS token generation. This led us to try a few different methods to prolong the output of our model, with each method improving upon the fallacies of the previous:

## Method 1: EOS Token Suppression
As the name suggests, our first method involved suppressing the EOS token. Normally, autoregressive transformers select the next token based off of the conditional probability given the previous tokens in the sequence. With this in mind, we used a sampling without replacement strategy to select the top 2 candidate tokens from the probability distribution. 
<br>
Interestingly enough, despite the physical suppression of the EOS token, the model tended to generate tokens that are semantically similar to the EOS token, such as “The End” or “Conclusion.” 
{insert photo of "The end." here}

This indicated that even though the model couldn’t physically end the output generation, it was semantically ending output by generating concluding phrases when the EOS token was unavailable. From this experiment we learned that the suppression of the EOS token alone may not be sufficient to prevent premature termination, and we need to guide the model towards producing more extended and coherent continuations. 

## Method 2: Modified Sampling Post-EOS Token
With the goal of creating a more stochastic sampling approach, we tried increasing the sampling temperature, and using techniques like top-k and nucleus sampling with higher thresholds. Although this allowed the model to continue generating tokens, the generated tokens after the “evicted” EOS token may diverge drastically from the previous context, which can result in incoherent or contextually irrelevant continuations.

{check doc for what to insert here}

Although a bit out of character, this is an example of a contextually irrelevant continuation. In the next methods, we attempt to resolve this issue.

## Method 3: Regenerating Tokens Prior to the EOS Token
Our 3rd method involves regenerating a portion of the sequence preceding the EOS token. This approach reintroduces part of the prior context, enabling the model to calibrate its understanding of the prompt and produce alternative continuations to align better with the original narrative output. 

We achieve this using: 1) a backstep hyperparameter, which determines the number of tokens to remove from the end of the generated sequence when the EOS token is predicted and 2) a KV-cache adjustment to reflect the truncated sequence. From here, the model resumes token generation from the truncated state, attempting to generate a different continuation without the influence of the previously predicted EOS token. 

This method did help avoid premature endings by allowing the model to explore alternative continuations. However, the loss of context by deleting the generated tokens still impacted the coherence and relevance of the generated text. 

## Method 4: Regenerating and Resampling Tokens Prior to the EOS token with Dynamic Temperature Adjustment
From our “Block-wise Analysis” section, we observe that entropy, varentropy, and information content all decrease in the block where the EOS token is generated. With this assumption that the model is converging towards generating the EOS token, we know that we need some way of introducing more randomness into the model. A way to achieve this is by increasing the temperature.

In this attempt we kept the backstep parameter and the KV-cache adjustment, but incorporated a *dynamic temperature adjustment* during the token regeneration. 

The generation resumes initially with a doubled sampling temperature to promote diversity in the immediate next token. This temperature is then gradually decreased using a scheduling function, aiming to return back to the original temperature at the point where the original EOS token was predicted. The model continues to generate tokens in this fashion until the output is complete.

{INSERT CODE BLOCK SHOWING TEMPERATURE INCREASE HERE}

The initial high temperature encourages the model to explore a wider range of possible continuations, reducing the likelihood of repeating the same ending. 

{INSERT CODE BLOCK SHOWING gradual temperature decrease hERE}

Gradually decreasing the temperature helps the model focus its prediction while slowly toning down on stochasticity. This helps *enhance the coherence of the generated text while maintaining a bit of randomness*.

The results of this method were by far the best out of our approaches. We were able to obtain 1,000 more generated tokens, and observed longer sequences with more contextually appropriate continuations compared to the previous methods. 
You can compare the results here {insert link to generated responses}. 

# Conclusion and Looking Ahead
Our experiments and analysis of EOS token generation highlight a few challenges and potential solutions in managing output length in large language models. Perpetual output is only useful for humans if it is *coherent and an actual extension of the original context*. Our first three methods showed that simply asking the transformer model to “try again” is not enough – we need to backtrack and introduce stochasticity to allow the model to think creatively on how to coherently extend output. 

Regenerating tokens prior to the EOS token showed promising results with a 1,000 token increase while balancing diversity and focus in the model’s predictions. Our results suggest that adaptive manipulation of the token generation process and sampling strategies can enhance the quality of generated text. 

We believe that further research into optimizing the scheduling function can be beneficial in prolonging output. Additional strategies to extend token generation earlier in the output can also be very beneficial towards having a longer output while maintaining focus, direction, and coherence. Currently our strategy only extends output in the latter areas of the original model response. Ideally, the model should be able to determine how long the ideal output should be, and generate output accordingly. 



